In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use all onehot-keepfirst nominal encoding for coupon type: Restaurant(20-50)

In [2]:
coupon_type = "Restaurant20To50"

In [3]:
encoding_type = 'onehot-keepfirst' ## use this string to access input file and name output files

os.makedirs(os.path.join('model', encoding_type), exist_ok=True)

In [4]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv(f'data/{coupon_type}_data_{encoding_type}.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 44.15% Yes
After SMOTE upsampling
	Num training samples: 1,329 (51.17% yes)
	Num testing samples:  333 (45.35% yes)


## Testing RandomForestClassifier with Grid Search Cross-validation


## RandomForestClassifier Parameters

The main parameters to adjust when using these methods:
* n_estimators
* max_features (sqrt is generally best default for classification. I have 55 features, so try values around 8) 

Make sure to include params:
* max_depth=None
* min_samples_split=2 
* oob_score=True (with bootstrapping)

In [5]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
#                 'param_min_samples_leaf', 
                'param_max_depth', 'param_max_leaf_nodes',
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [6]:
param_grid={'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            'min_samples_split': [2, 3],
            'n_estimators': [100, 150, 200],
#             'min_samples_leaf': [1, 2, 3, 4, 5, 10],
            'max_depth': [None, 20, 25, 30],
            'max_leaf_nodes': [None, 100, 200]
           }

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [None, 20, 25, 30],
                         'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15],
                         'max_leaf_nodes': [None, 100, 200],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_features=6, min_samples_split=3, n_estimators=200,
                       oob_score=True, random_state=42)

In [7]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.7193378480060195
ROC AUC: TEST 0.804599



In [8]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,param_max_depth,param_max_leaf_nodes,mean_test_score,std_test_score,mean_train_score,std_train_score
23,1,6,3,200,None,None,0.78320355,0.01088974,0.99998973,0.00000493
419,2,6,3,200,25,None,0.78317251,0.01131275,0.99998973,0.00000564
486,3,10,2,100,25,None,0.78298191,0.01047576,0.99999327,0.00000518
617,4,6,3,200,30,None,0.78276334,0.01286221,0.99998903,0.00000518
290,5,10,2,200,20,None,0.78242410,0.01419055,0.99999327,0.00000518
615,6,6,3,100,30,None,0.78222737,0.01040146,0.99998194,0.00001128
200,7,5,2,200,20,None,0.78211876,0.01242680,0.99999327,0.00000518
21,8,6,3,100,None,None,0.78211460,0.01034986,0.99998336,0.00000843
687,9,10,3,100,30,None,0.78201252,0.01331863,0.99998124,0.00001154
93,10,10,3,100,None,None,0.78179277,0.01204833,0.99998054,0.00001337


In [9]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
26,expiration_2h,0.02681498
25,expiration_1d,0.02057484
87,CoffeeHouse_1~3,0.01803474
107,Restaurant20To50_never,0.01780095
106,Restaurant20To50_less1,0.01699168
94,CarryAway_4~8,0.01690440
93,CarryAway_1~3,0.01675993
43,education_Bachelors degree,0.01639364
98,RestaurantLessThan20_1~3,0.01631688
86,Bar_never,0.01613481


In [10]:
import pickle

with open(f'model/{encoding_type}/{coupon_type}_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open(f'model/{encoding_type}/{coupon_type}_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)

In [11]:
## save train/test info to file
Xy_tt = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

for k in Xy_tt.keys():
    fname = f'data/{coupon_type}_{k}_{encoding_type}.csv'
    print(f'Saving file: {fname}')
    Xy_tt[k].to_csv(fname, index=False)

Saving file: data/Restaurant20To50_X_train_onehot-keepfirst.csv
Saving file: data/Restaurant20To50_X_test_onehot-keepfirst.csv
Saving file: data/Restaurant20To50_y_train_onehot-keepfirst.csv
Saving file: data/Restaurant20To50_y_test_onehot-keepfirst.csv
